### ISSUE 및 필요한 작업

1. instrument 파일의 ENDDATE 정보가 NECIS의 실제 데이터와 오차  
2. PMNM 관측소 정보가 (2020년 이후 운영종료한 관측소) instrument 파일 내에 존재하지 않음 (임시 해결) 
3. DEPTH 컬럼 의미

---

1. 과정 2에서 이미 데이터가 instrument안에 있을 경우  
2. NECIS에서 비슷하거나 같은 이름의 관측소가 존재할 경우    
3. 스크래핑 if문 수정필요  
4. 전체적으로 예외항목(ex.PMNM)에 대비해서 코드를 정리해야 할 필요가 있음 / 모듈화를 해야할 듯

---

### 필요한 모듈 및 데이터파일 불러오기

In [100]:
import numpy as np
import pandas as pd
import os
import datetime
from datetime import date
import re

from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import time

In [101]:
# 수정의 대상이 될 korea_instrument 파일을 불러옵니다.

inst = pd.read_csv('korea_instrument.list', sep='\s{1,}', engine='python')
inst['LAT'] = inst['LAT'].astype('float64').apply(lambda x:'%.6f'% x)
inst['LON'] = inst['LON'].astype('float64').apply(lambda x:'%.6f'% x)
inst['CMPAZ'] = inst['CMPAZ'].astype('str')
inst['CMPINC'] = inst['CMPINC'].astype('str')
inst['ONDATE'] = inst['ONDATE'].astype('str')
inst['SP'] = inst['SP'].astype('str')
inst.head(5)

,#NET,STA,LAT,LON,ELEV,DEP,CHAN,CMPAZ,CMPINC,ONDATE,ENDDATE,SP,SENSOR,RECORDER
0,KG,AJD,34.747890,126.123840,125,0,ELZ,0,-90,201107270000,260000000000,100,CMG-40T-1,Q330HRS
1,KG,AJD,34.747890,126.123840,125,0,ELN,0,0,201107270000,260000000000,100,CMG-40T-1,Q330HRS
2,KG,AJD,34.747890,126.123840,125,0,ELE,90,0,201107270000,260000000000,100,CMG-40T-1,Q330HRS
3,KG,AJD,34.747890,126.123840,125,0,SLZ,0,-90,201107270000,260000000000,20,CMG-40T-1,Q330HRS
4,KG,AJD,34.747890,126.123840,125,0,SLN,0,0,201107270000,260000000000,20,CMG-40T-1,Q330HRS


In [102]:
# 관측소 폐쇄 & 신설 정보 들어있는 xlsx파일을 불러옵니다.
# Excel에서 멀티컬럼명에 빈칸이 있을 시 pd.read_excel을 이용하면 파이썬에서 'Unnamed... ' 로 출력 -> 보기 좋게 List comprehension으로 빈칸으로 바꿔줌 
# Excel에서 datetime을 파이썬으로 그냥 불러올 시 1900-01-01 기준으로 경과한 일수의 포맷으로 반환되기 때문에, pd.to_datetime을 이용해 정상적으로 포맷팅해줌
# value중 NaN값은 빈 칸으로 대체했습니다

ks_stop = pd.read_excel('./xlsx/KS_stop.xlsx', engine='openpyxl', header = [1,2])
[ks_stop.rename( columns={'Unnamed: ' + str(i) + '_level_1':' '}, inplace=True) for i in range(11)]
ks_stop['관측시작일'] = pd.to_datetime(ks_stop['관측시작일', ' '], unit='d', origin='1899-12-30')
ks_stop['관측종료일'] = pd.to_datetime(ks_stop['관측종료일', ' '], unit='d', origin='1899-12-30')
ks_stop = ks_stop.fillna('')

ks_run = pd.read_excel('./xlsx/KS_run.xlsx', engine='openpyxl', header = [1,2])
[ks_run.rename( columns={'Unnamed: ' + str(i) + '_level_1':' '}, inplace=True) for i in range(11)]
ks_run['관측시작일'] = pd.to_datetime(ks_run['관측시작일', ' '], unit='d', origin='1899-12-30')
ks_run['관측종료일'] = pd.to_datetime(ks_run['관측종료일', ' '], unit='d', origin='1899-12-30')
ks_run = ks_run.fillna('')

kg_stop = pd.read_excel('./xlsx/KG_stop.xlsx', engine='openpyxl', header = [1,2])
[kg_stop.rename( columns={'Unnamed: ' + str(i) + '_level_1':' '}, inplace=True) for i in range(11)]
kg_stop['관측시작일'] = pd.to_datetime(kg_stop['관측시작일', ' '], unit='d', origin='1899-12-30')
kg_stop['관측종료일'] = pd.to_datetime(kg_stop['관측종료일', ' '], unit='d', origin='1899-12-30')
kg_stop = kg_stop.fillna('')

kg_run = pd.read_excel('./xlsx/KG_run.xlsx', engine='openpyxl', header = [1,2])
[kg_run.rename( columns={'Unnamed: ' + str(i) + '_level_1':' '}, inplace=True) for i in range(11)]
kg_run['관측시작일'] = pd.to_datetime(kg_run['관측시작일', ' '], unit='d', origin='1899-12-30')
kg_run['관측종료일'] = pd.to_datetime(kg_run['관측종료일', ' '], unit='d', origin='1899-12-30')
kg_run = kg_run.fillna('')

# 데이터프레임 4개 병합하기
df = pd.concat([ks_run, ks_stop, kg_run, kg_stop], ignore_index = True)
df.head(5)

관측망코드  지점코드 지점코드명(한)   지점코드명(영) 관측지점         지점종류      관측시작일 관측종료일  \
                                                                       
0    KS  ADOA       송현  Songhyeon   송현          가속도 2015-09-30   NaT   
1    KS  AGSA       앙성   Angseong   앙성          가속도 2018-03-09   NaT   
2    KS   AMD      안마도     Anmado  안마도  속도+광대역, 가속도 2015-09-30   NaT   
3    KS  ANDB       안덕     Andeok   안덕  속도+광대역, 가속도 2022-11-29   NaT   
4    KS  ANHA       안흥    Anheung   안흥          가속도 2022-11-29   NaT   

      위도(ºN)      경도(ºE) 상태정보:대                        장비정보  \
                                                         속도   
0  36.572600  128.700900   운영상태                           -   
1  37.091700  127.808000   운영상태                           -   
2  35.343600  126.030000   운영상태       STS-2.5-A(지표형 광대역 속도)   
3  33.256580  126.328750   운영상태  Trilium120PH-A(시추형 광대역 속도)   
4  37.464696  128.155024   운영상태                           -   

                                     
                       가속도      기록계  
0     ES-DH-A(시추형 광대역 가속도)  Q330HRS  
1     ES-DH-A(시추형 광대역 가속도)   Q330SP  
2      ES-T-A(지표형 광대역 가속도)  Q330HRS  
3  Titan-PH-A(시추형 광대역 가속도)  CENTAUR  
4  Titan-PH-A(시추형 광대역 가속도)  CENTAUR

In [103]:
df

관측망코드  지점코드     지점코드명(한)                                 지점코드명(영)  \
                                                                        
0      KS  ADOA           송현                                Songhyeon   
1      KS  AGSA           앙성                                 Angseong   
2      KS   AMD          안마도                                   Anmado   
3      KS  ANDB           안덕                                   Andeok   
4      KS  ANHA           안흥                                  Anheung   
..    ...   ...          ...                                      ...   
540    KG   YPD          연평도                             Yeonpyeongdo   
541    KG   YSB           양산                                  Yangsan   
542    KG  YSUK  연세대학교 국제캠퍼스  Yeonsei University International Campus   
543    KG  YSUM  연세대학교 미래캠퍼스          Yeonsei University Mirae Campus   
544    KG  GKP1          경북대                             Gyeongbukdae   

            관측지점         지점종류      관측시작일      관측종료일     위도(ºN)      경도(ºE)  \
                                                                             
0             송현          가속도 2015-09-30        NaT  36.572600  128.700900   
1             앙성          가속도 2018-03-09        NaT  37.091700  127.808000   
2            안마도  속도+광대역, 가속도 2015-09-30        NaT  35.343600  126.030000   
3             안덕  속도+광대역, 가속도 2022-11-29        NaT  33.256580  126.328750   
4             안흥          가속도 2022-11-29        NaT  37.464696  128.155024   
..           ...          ...        ...        ...        ...         ...   
540          연평도  속도+단주기, 가속도 2006-11-13        NaT  37.661510  125.699050   
541           양산  속도+광대역, 가속도 2009-02-06        NaT  35.468110  129.079920   
542  연세대학교 국제캠퍼스          가속도 2019-12-24        NaT  37.379200  126.671000   
543  연세대학교 미래캠퍼스          가속도 2019-12-24        NaT  37.282600  127.903400   
544          경북대              1998-10-30 2019-01-25  35.889300  128.606050   

    상태정보:대                        장비정보                                    
                                    속도                      가속도      기록계  
0     운영상태                           -     ES-DH-A(시추형 광대역 가속도)  Q330HRS  
1     운영상태                           -     ES-DH-A(시추형 광대역 가속도)   Q330SP  
2     운영상태       STS-2.5-A(지표형 광대역 속도)      ES-T-A(지표형 광대역 가속도)  Q330HRS  
3     운영상태  Trilium120PH-A(시추형 광대역 속도)  Titan-PH-A(시추형 광대역 가속도)  CENTAUR  
4     운영상태                           -  Titan-PH-A(시추형 광대역 가속도)  CENTAUR  
..     ...                         ...                      ...      ...  
540   운영상태            CMG-40T-1-A(단주기)      ES-T-A(지표형 광대역 가속도)  Q330HRS  
541   운영상태       CMG-3TB-C(시추형 광대역 속도)     ES-DH-A(시추형 광대역 가속도)  Q330HRS  
542   운영상태                           -     ES-DH-A(시추형 광대역 가속도)  Q330HRS  
543   운영상태                           -     ES-DH-A(시추형 광대역 가속도)  Q330HRS  
544   종료상태                           -                        -        -  

[545 rows x 14 columns]

---
### 1. 2020-10-05이후 폐쇄된 관측소 데이터의 ENDTIME value 수정하기

In [104]:
# 2020-10-05 이후 폐쇄된 관측소명 리스트 저장

stop_value = df.loc[df['관측종료일', ' '] >= '2020-10-05']['지점코드', ' '].values
stop_value

array(['ADO2', 'CEJA', 'CHC2', 'DACB', 'DAG2', 'DGY2', 'EMSB', 'GOS2',
       'HALB', 'HUK2', 'HWCB', 'ISGB', 'JEO2', 'KOJ2', 'MIYA', 'OKEB',
       'PHA2', 'PMNM', 'SES2', 'SHHB', 'SMKB', 'ULJ2', 'WJU2', 'YAPA'],
      dtype=object)

In [105]:
# 2020-10-05 이후 폐쇄된 관측소의 관측종료일 포맷형태를 YYYY-mm-dd에서 YYmmdd0000으로 변경 -> stopdate 변수에 저장
# instrument dataframe의 수정이 필요한 ENDTIME value를 위에서 변환한 형태의 stopdate로 대체합니다
# PMNM 관측소 데이터가 instrument 파일에 존재하지 않기 때문에, 이후 작업에 용이하게끔 exception 변수에 입력하였습니다.

for i in stop_value: 
    stopdate = pd.Timestamp(df.loc[df['지점코드', ' '] == i]['관측종료일', ' '].values[0]).strftime('%Y%m%d0000')
    arr = inst.loc[(inst.STA == i) & (inst.ENDDATE == 260000000000)]['ENDDATE'].values
    if len(arr) == 0:
        exception = i
        continue
    else :
        for idx in range(len(arr)):
            inst.loc[(inst.STA == i) & (inst.ENDDATE == 260000000000)] = inst.loc[(inst.STA == i) & (inst.ENDDATE == 260000000000)].replace(260000000000, stopdate)

---
### 2. 2020-10-05 이후 추가된 관측소의 데이터를 데이터프레임화 


In [106]:
# 2020-10-05 이후 추가된 관측소명 리스트 저장
# instrument에 해당 관측소 정보가 이미 있는지 여부 체크

new_value = df.loc[df['관측시작일', ' '] >= '2020-10-05']['지점코드', ' '].values
new_value = np.append(new_value, exception)
new_value

array(['ANDB', 'ANHA', 'BOHB', 'CDSA', 'CE2A', 'CEBA', 'CHIA', 'DAYA',
       'DDCB', 'GEJA', 'GIJA', 'GOBA', 'GOSF', 'HA2B', 'HSJB', 'HYDA',
       'INJA', 'IPTA', 'ISHB', 'IYIB', 'IYPA', 'JAEB', 'JDGB', 'JEDA',
       'JGJA', 'JHCB', 'JJBA', 'JOGB', 'JWJB', 'KESB', 'MSMB', 'NGJA',
       'NHPA', 'NHSB', 'NPGA', 'NSJA', 'NYSA', 'OADB', 'OGSB', 'OHBA',
       'OJSA', 'OOJB', 'OPHB', 'OPSA', 'OSBA', 'OUJA', 'OWSA', 'PHDA',
       'PMYA', 'PTYC', 'PUYA', 'SEJA', 'SENA', 'SNWA', 'WAJB', 'GHR',
       'OJR', 'PMNM'], dtype=object)

In [107]:
inst_value = inst.STA.unique()
inst_value

array(['AJD', 'BBK', 'BGD', 'BOG', 'BRD', 'BRN', 'BRS', 'CGD', 'CGU',
       'CHJ', 'CHNB', 'CHS', 'CRB', 'DES', 'DKJ', 'DNH', 'DOKDO', 'DUC',
       'GCN', 'GKP1', 'GKP2', 'GRE', 'GSU', 'HAK', 'HCH', 'HDB', 'HKU',
       'HSB', 'HWSB', 'IBA', 'JJB', 'JRB', 'JSB', 'JUC', 'KHD', 'KIP',
       'KJM', 'KMC', 'KNUC', 'KNUD', 'KRA', 'KRB', 'KRC', 'KRN', 'KSA',
       'MAK', 'MGB', 'MKL', 'MRD', 'MUN', 'NPR', 'NSN', 'OKC', 'PCH',
       'PKNU', 'POHB', 'POSB', 'SIG', 'SND', 'SNU', 'SRJ', 'TJN', 'UCN',
       'UJA', 'UNI', 'WDL', 'WID', 'WSA', 'WSB', 'WSC', 'WSD', 'WSN',
       'YGA', 'YGB', 'YGN', 'YIN', 'YKB', 'YNB', 'YNG', 'YPD', 'YSB',
       'YSUK', 'YSUM', 'ADO2', 'ADO', 'ADOA', 'AGSA', 'AMD', 'AND', 'ANM',
       'ANMA', 'ASNA', 'BAR2', 'BAR', 'BAU', 'BGDB', 'BKWA', 'BLGA',
       'BLLA', 'BOE', 'BOGA', 'BON2', 'BON', 'BOSB', 'BSA', 'BSAA',
       'BURB', 'BUS2', 'BUS3', 'BUS', 'BUSA', 'BUY', 'BUYB', 'CEA2',
       'CEA', 'CEJ', 'CEJA', 'CGAA', 'CGDA', 'CGIA', 'CGPA', 'CGUB',
       'C

In [108]:
# 두 리스트의 교집합을 출력 -> 없음

list(set(new_value) & set(inst_value))

[]

In [109]:
# 2020-10-05 이후 추가된 관측소 데이터들로 포맷팅 이후 instrument 파일에 추가될 데이터 목록

df_new = df.loc[df['관측시작일', ' '] >= '2020-10-05']
df_new = pd.concat([df_new, df.loc[df['지점코드', ' '] == exception]])
df_new.head(5)

관측망코드  지점코드 지점코드명(한)  지점코드명(영) 관측지점         지점종류      관측시작일 관측종료일  \
                                                                       
3     KS  ANDB       안덕    Andeok   안덕  속도+광대역, 가속도 2022-11-29   NaT   
4     KS  ANHA       안흥   Anheung   안흥          가속도 2022-11-29   NaT   
14    KS  BOHB       봉화   Bonghwa   봉화  속도+광대역, 가속도 2022-11-29   NaT   
22    KS  CDSA       달성  Dalseong   달성          가속도 2021-11-05   NaT   
23    KS  CE2A       청주  Cheongju   청주          가속도 2020-12-24   NaT   

       위도(ºN)      경도(ºE) 상태정보:대                        장비정보  \
                                                          속도   
3   33.256580  126.328750   운영상태  Trilium120PH-A(시추형 광대역 속도)   
4   37.464696  128.155024   운영상태                           -   
14  36.926326  129.053685   운영상태  Trilium120PH-A(시추형 광대역 속도)   
22  35.704465  128.444361   운영상태                           -   
23  36.626800  127.513500   운영상태                           -   

                                      
                        가속도      기록계  
3   Titan-PH-A(시추형 광대역 가속도)  CENTAUR  
4   Titan-PH-A(시추형 광대역 가속도)  CENTAUR  
14  Titan-PH-A(시추형 광대역 가속도)  CENTAUR  
22  Titan-PH-A(시추형 광대역 가속도)  CENTAUR  
23     ES-DH-A(시추형 광대역 가속도)  Q330HRS

---
### 3. Web Scraping을 이용해 NECIS로부터 해발고도, 채널 및 기록계 & 센서 모델 정보 추출 후 df_new에 넣어주기

In [12]:
# NECIS로부터 필요한 데이터 긁어오기
# Selenium moduie 및 경로 내에 chromedriver.exe 존재해야 합니다 (https://wikidocs.net/91474)
# 2번 과정에서 만든 df_new는 관측소의 각 채널정보와 기록계 및 센서 모델이 누락되어있기 때문에 해당 과정에서 스크랩해줍니다.
# NoSuchElementException 에러 발생 시 메모리 확보 or time.sleep(n) 조절 

raw_data = pd.DataFrame(columns = ['STA', 'ELEV', 'CHAN', 'SENSOR', 'RECORD'])

try: 

    options = wd.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--blink-settings=imagesEnabled=false')
    driver = wd.Chrome(executable_path = 'chromedriver.exe', options = options)

    # NECIS 로그인
    
    driver.get('https://necis.kma.go.kr/necis-dbf/user/common/userLoginNewForm.do')
    element_id = driver.find_element(By.ID, 'email')
    element_id.send_keys('NECIS ID를 입력하세요')
    element_pwd = driver.find_element(By.ID,'pPasswd')
    element_pwd.send_keys('NECIS PASSWORD를 입력하세요')
    time.sleep(3)
    driver.find_element(By.CLASS_NAME, 'btn_login').click()
    time.sleep(3)

    # 지진관측소 페이지 이동 및 데이터 추출
    # 관측망코드 콤보박스의 Default값이 KS : 기상청으로 설정되어져있고, KG : 지자연데이터를 구하기 위해서 if문을 추가하였습니다.
    # PMNM 데이터가 기존 instrument 파일 내 존재하지 않기 때문에, 운영상태 콤보박스의 값을 ENDMT으로 설정한 후 해당 관측소 데이터를 추출하였습니다.
    
    driver.get('https://necis.kma.go.kr/necis-dbf/user/ob/earthquakeObservatoryListPage.do?fromFlag=start')
    
    for i in new_value:
        if (i == 'GHR') or (i == 'OJR'): 
            select = Select(driver.find_element(By.NAME, 'spotNetTypeFilter'))
            select.select_by_value('KG')
        if i == 'PMNM':
            select = Select(driver.find_element(By.NAME, 'spotUseYnFilter'))
            select.select_by_value('ENDMT')
        time.sleep(1)
        searchbox = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/form/div/span[2]/span/input')
        searchbox.send_keys(i)
        time.sleep(1)
        searchbox.send_keys('\n')
        time.sleep(1)
        sta_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/table/tbody/tr/td[3]').click()
        sta = driver.find_element(By.ID, "spotSta").get_attribute('value')
        elev = driver.find_element(By.ID, "spotElev").get_attribute('value')
        
        for i in range(1, 50): 
            try:
                new_data = pd.DataFrame(columns = ['STA', 'ELEV', 'CHAN', 'SENSOR', 'RECORD'])
                time.sleep(1)
                chan = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[2]/div/div[2]/div[4]/div/div/table/tbody/tr['+str(i)+']/td[2]/input').get_attribute('value')
                sensor = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[2]/div/div[2]/div[4]/div/div/table/tbody/tr['+str(i)+']/td[8]/input').get_attribute('value')
                recorder = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[2]/div/div[2]/div[4]/div/div/table/tbody/tr['+str(i)+']/td[6]/input').get_attribute('value')
                new_data.loc[0] = [sta, elev, chan, sensor, recorder]
                raw_data = pd.concat([raw_data, new_data], ignore_index = True)
                time.sleep(1)
            except:
                break
            
        driver.get('https://necis.kma.go.kr/necis-dbf/user/ob/earthquakeObservatoryListPage.do?fromFlag=start')
    
    raw_data.to_csv('scraping.csv', index = False)
    
except Exception:
    raise

finally:
    if driver is not None:
        driver.quit()

C:\Users\neimb\AppData\Local\Temp/ipykernel_7720/210497256.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome(executable_path = 'chromedriver.exe', options = options)


In [110]:
# 이전 과정에서 만든 csv파일을 불러옵니다.

scraping = pd.read_csv('scraping.csv', sep=',', engine='python')
scraping.head(5)

,STA,ELEV,CHAN,SENSOR,RECORD
0,ANDB,84.0,BGE,Titan-PH-A,Centaur
1,ANDB,84.0,BGN,Titan-PH-A,Centaur
2,ANDB,84.0,BGZ,Titan-PH-A,Centaur
3,ANDB,84.0,BHE,Trilium120PH-A,Centaur
4,ANDB,84.0,BHN,Trilium120PH-A,Centaur


---
### 4. CMPAZ, CMPINC, SP 조건문 만들어서 새로운 관측소에 넣어주기 & instrument에 넣어주기 위한 포맷팅 작업

In [111]:
# df_new 데이터프레임에서 포맷팅에 필요한 컬럼만 뽑아온 후, 멀티컬럼을 제거해주었습니다.

df_n = df_new.loc[:, ['관측망코드', '지점코드', '위도(ºN)', '경도(ºE)', '관측시작일', '관측종료일' ]]
df_n.columns = df_n.columns.droplevel(1)

df_n.head(5)

,관측망코드,지점코드,위도(ºN),경도(ºE),관측시작일,관측종료일
3,KS,ANDB,33.256580,126.328750,2022-11-29,NaT
4,KS,ANHA,37.464696,128.155024,2022-11-29,NaT
14,KS,BOHB,36.926326,129.053685,2022-11-29,NaT
22,KS,CDSA,35.704465,128.444361,2021-11-05,NaT
23,KS,CE2A,36.626800,127.513500,2020-12-24,NaT


In [112]:
# scraping & df_n의 ['STA']/['지점코드'] 를 기준으로 두 데이터프레임을 병합해주었으며, instrument 파일에 들어갈 양식으로 컬럼명 & 컬럼 순서 & 데이터 형식을 변경해주었습니다.

data = pd.merge(scraping, df_n, left_on = 'STA', right_on = '지점코드')
data = data.rename(columns={'관측망코드':'#NET', '위도(ºN)':'LAT', '경도(ºE)':'LON', '관측시작일':'ONDATE', '관측종료일':'ENDDATE', 'RECORD':'RECORDER'})

data['DEP'] = 0
data['CMPAZ'] = ''
data['CMPINC'] = ''
data['SP'] = ''

data['LAT'] = data['LAT'].astype('float64').apply(lambda x:'%.6f'% x)
data['LON'] = data['LON'].astype('float64').apply(lambda x:'%.6f'% x)
data['ELEV'] = data['ELEV'].astype('float64').astype('int64')
data['ONDATE'] = data['ONDATE'].apply(lambda x : datetime.datetime.strftime(x,'%Y%m%d0000'))
data['ENDDATE'] = '260000000000'

stopdate = pd.Timestamp(df.loc[df['지점코드', ' '] == exception]['관측종료일', ' '].values[0]).strftime('%Y%m%d0000')
data.loc[data['STA'].str.match(exception), 'ENDDATE'] = stopdate

data.loc[data['CHAN'].str.endswith('Z'), 'CMPAZ'] = '0'
data.loc[data['CHAN'].str.endswith('N'), 'CMPAZ'] = '0'
data.loc[data['CHAN'].str.endswith('E'), 'CMPAZ'] = '90'
data.loc[data['CHAN'].str.endswith('Z'), 'CMPINC'] = '-90'
data.loc[data['CHAN'].str.endswith('N'), 'CMPINC'] = '0'
data.loc[data['CHAN'].str.endswith('E'), 'CMPINC'] = '0'

data.loc[data['CHAN'].str.match('HH'), 'SP'] = '100'
data.loc[data['CHAN'].str.match('HG'), 'SP'] = '100'
data.loc[data['CHAN'].str.match('EL'), 'SP'] = '100'
data.SP = data.SP.replace('', '20')
data['SENSOR'] = data['SENSOR'].apply(lambda x : 'TITAN-PH(A)' if x == 'Titan-PH-A' 
                                      else  ('TRILIUM120PH(A)' if x == 'Trilium120PH-A' 
                                             else ('ES-DH(A)' if x == 'ES-DH-A' 
                                                   else ('ES-T(A)' if x == 'ES-T-A' 
                                                         else ('CMG-40T-1(B)' if x == 'CMG-40T-1-B' 
                                                               else ('CMG-3TB(B)' if x == 'CMG-3TB-B' 
                                                                     else ('3ESPC' if x == '3ESPC'
                                                                           else 'CMG-40T-1(A)')))))))
data.RECORDER = data['RECORDER'].apply(lambda x : str.upper(x))
data = data[['#NET', 'STA', 'LAT', 'LON', 'ELEV', 'DEP', 'CHAN', 'CMPAZ', 'CMPINC', 'ONDATE', 'ENDDATE', 'SP', 'SENSOR', 'RECORDER']]
data.head(5)

,#NET,STA,LAT,LON,ELEV,DEP,CHAN,CMPAZ,CMPINC,ONDATE,ENDDATE,SP,SENSOR,RECORDER
0,KS,ANDB,33.256580,126.328750,84,0,BGE,90,0,202211290000,260000000000,20,TITAN-PH(A),CENTAUR
1,KS,ANDB,33.256580,126.328750,84,0,BGN,0,0,202211290000,260000000000,20,TITAN-PH(A),CENTAUR
2,KS,ANDB,33.256580,126.328750,84,0,BGZ,0,-90,202211290000,260000000000,20,TITAN-PH(A),CENTAUR
3,KS,ANDB,33.256580,126.328750,84,0,BHE,90,0,202211290000,260000000000,20,TRILIUM120PH(A),CENTAUR
4,KS,ANDB,33.256580,126.328750,84,0,BHN,0,0,202211290000,260000000000,20,TRILIUM120PH(A),CENTAUR


In [113]:
# 기존 instrument 데이터에 이전 단계에서 만든 새로운 관측소 데이터 & 누락 데이터를 병합시켜주었으며, sort_values를 이용해 관측망코드 - 관측소명 - 관측시작일시의 순으로 정렬해주었습니다.

chan_dict = {'ELZ': 0, 'ELN': 1, 'ELE': 2, 'SLZ' : 3, 'SLN' : 4, 'SLE' : 5, 'HGZ' : 6, 'HGN' : 7, 'HGE' : 8, 'BGZ' : 9, 'BGN' : 10, 'BGE' :11\
         ,'HHZ' : 12, 'HHN' : 13, 'HHE' : 14, 'BHZ' : 15, 'BHN' : 16, 'BHE' : 17, 'LHZ' : 18, 'LHN' : 19, 'LHE' : 20, 'HAZ' : 21, 'HAN' : 22, 'HAE' : 23}
l_list = ['ELZ', 'ELN', 'ELE', 'SLZ', 'SLN', 'SLE']
g_list = ['HGZ', 'HGN', 'HGE', 'BGZ', 'BGN', 'BGE']
h_list = ['HHZ', 'HHN', 'HHE', 'BHZ', 'BHN', 'BHE', 'LHZ', 'LHN', 'LHE']
a_list = ['HAZ', 'HAN', 'HAE']
                         
result = pd.concat([inst, data], ignore_index = True)
result['rank_l'] = result['CHAN'].map(chan_dict)
result['rank_n'] = result['CHAN'].apply(lambda x : 0 if x in l_list else (1 if x in g_list else (2 if x in h_list else (3))))

result = result.sort_values(by=['#NET','STA','rank_n','ONDATE','ENDDATE','rank_l'], ignore_index = True)
result = result.drop(['rank_l', 'rank_n'], axis = 1)
result.head(5)

,#NET,STA,LAT,LON,ELEV,DEP,CHAN,CMPAZ,CMPINC,ONDATE,ENDDATE,SP,SENSOR,RECORDER
0,KG,AJD,34.747890,126.123840,125,0,ELZ,0,-90,201107270000,260000000000,100,CMG-40T-1,Q330HRS
1,KG,AJD,34.747890,126.123840,125,0,ELN,0,0,201107270000,260000000000,100,CMG-40T-1,Q330HRS
2,KG,AJD,34.747890,126.123840,125,0,ELE,90,0,201107270000,260000000000,100,CMG-40T-1,Q330HRS
3,KG,AJD,34.747890,126.123840,125,0,SLZ,0,-90,201107270000,260000000000,20,CMG-40T-1,Q330HRS
4,KG,AJD,34.747890,126.123840,125,0,SLN,0,0,201107270000,260000000000,20,CMG-40T-1,Q330HRS


In [114]:
# instrument 파일의 포맷에 맞게 포매팅을 진행해주었으며, 코드를 실행한 날짜로 파일 제목을 설정해주었습니다.

with open('korea_instrument_' + datetime.datetime.today().strftime('%Y%m%d') + '.list', 'w') as f:
    columns = '{0}{1:>4}{2:>10}{3:>12}{4:>6}{5:>6}{6:>6}{7:>6}{8:>7}{9:>12}{10:>13}{11:>6}{12:>16}{13:>16}'.format(result.columns[0], result.columns[1], result.columns[2], result.columns[3], result.columns[4], result.columns[5],
                                                                                                                   result.columns[6], result.columns[7], result.columns[8], result.columns[9], result.columns[10], result.columns[11],
                                                                                                                   result.columns[12], result.columns[13])
    f.write(columns + '\n')
    for i in range(len(result)):
        lines = '{0}{1:>6}{2:>10}{3:>12}{4:>6}{5:>6}{6:>6}{7:>6}{8:>6}{9:>13}{10:>13}{11:>6}{12:>16}{13:>16}'.format(result.loc[i][0], result.loc[i][1], result.loc[i][2], result.loc[i][3], result.loc[i][4], result.loc[i][5],
                                                                                                                    result.loc[i][6], result.loc[i][7], result.loc[i][8], result.loc[i][9], result.loc[i][10], result.loc[i][11],
                                                                                                                    result.loc[i][12], result.loc[i][13])
        f.write(lines + '\n')